In [3]:
!pip install finnhub-python akshare tushare parsel requests pandas tqdm pytz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 20.2 MB/s eta 0:00:00
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5615 sha256=e4ae7fe50e325c7f0e7e5b94646677f69f13f6ad3380170658a635468b04ab81
  Stored in directory: /root/.cache/pip/wheels/a2/e3/6a/59f241672f3221253b1236359b0a711dabb69795a006184398
Successfully built jsonpath
  Attempting uninstall:

In [7]:
from google.colab import userdata
finnhub_api_key = userdata.get('FINNHUB_API_KEY')

In [8]:
import finnhub

finnhub_client = finnhub.Client(api_key=finnhub_api_key)

In [10]:
import requests
import parsel
from lxml import etree
from tqdm import tqdm
import time
import re

def check_china_ips(proxies_list):
    """检测ip的方法"""
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'}

    can_use = []
    for proxy in tqdm(proxies_list, desc = "Checking ips"):
        try:
            response = requests.get('http://www.baidu.com', headers=headers, proxies=proxy, timeout=1)  # 超时报错
            if response.status_code == 200:
                can_use.append(proxy)
        except Exception as error:
            # print(error)
            pass
    return can_use

def check_us_ips(proxies_list):
    """检测ip的方法"""
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'}

    can_use = []
    for proxy in tqdm(proxies_list, desc = "Checking ips"):
        try:
            response = requests.get('http://www.google.com', headers=headers, proxies=proxy, timeout=1)  # 超时报错
            if response.status_code == 200:
                can_use.append(proxy)
        except Exception as error:
            # print(error)
            pass
    return can_use

def get_china_free_proxy(pages = 10):
    proxies_list = []
    for page in tqdm(range(1, pages+1), desc = "Gathering free ips by pages..."):

        base_url = f'https://www.kuaidaili.com/free/inha/{page}'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'}
        success = False
        while not success:
            try:
                response = requests.get(base_url, headers=headers)
                data = response.text
                res = etree.HTML(data)
                trs = res.xpath('//table/tbody/tr')
                if len(trs)!=0:
                    success = True
                    for tr in trs:
                        proxies_dict = {}
                        http_type = tr.xpath('./td[4]/text()')[0]
                        ip_num = tr.xpath('./td[1]/text()')[0]
                        port_num = tr.xpath('./td[2]/text()')[0]
                        proxies_dict[http_type] = ip_num + ':' + port_num
                        proxies_list.append(proxies_dict)
                else:
                    time.delay(0.01)

            except:
                pass

    can_use = check_china_ips(proxies_list)

    print(f'获取到的代理ip数量: {len(proxies_list)} 。Get proxy ips: {len(proxies_list)}.')
    print(f'能用的代理数量： {len(can_use)}。Usable proxy ips: {len(can_use)}.' )

    return can_use

def get_us_free_proxy(pages = 10):
    url = "https://openproxy.space/list/http"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Connection Error. Please make sure that your computer now have the access to Google.com")
    res = etree.HTML(response.text)
    http_type = "HTTP"
    proxies_list = []

    scripts = res.xpath("//script")
    content = scripts[3].xpath(".//text()")
    pattern = re.compile('LIST",data:(.+),added:')
    result_list = pattern.findall(content[0])
    result_list = result_list[0].strip("[{").strip("}]").split("},{")

    for result in result_list:
        pattern = re.compile('\[(.+)\]')
        result = pattern.findall(result)
        result = result[0].split(",")
        result = [r.strip("\"") for r in result]
        for ip in result:
            proxies_list.append(
                {http_type: ip}
            )
    total = pages* 15
    proxies_list = proxies_list[:total]
    can_use = check_us_ips(proxies_list)
    print(f'Get proxy ips: {len(proxies_list)}.')
    print(f'Usable proxy ips: {len(can_use)}.' )

    return can_use

class Kuaidaili:
    def __init__(self, tunnel, username, password):
        self.tunnel = tunnel
        self.username = username
        self.password = password

    def get_kuaidaili_tunnel_proxy(self):
        proxies = {
            "http": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": self.username, "pwd": self.password, "proxy": self.tunnel},
            "https": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": self.username, "pwd": self.password, "proxy": self.tunnel}
        }
        return proxies

In [11]:
import warnings
warnings.filterwarnings("ignore")

import requests

class FinNLP_Downloader:
    def __init__(self, args = {}):
        self.use_proxy = True if "use_proxy" in args.keys() else False
        if self.use_proxy:
            self.country = args["use_proxy"]
        else:
            self.country = None
        self.max_retry = args["max_retry"] if "max_retry" in args.keys() else 1
        self.proxy_pages = args["proxy_pages"] if "proxy_pages" in args.keys() else 5
        if self.use_proxy:
            if "kuaidaili" in self.country:
                # tunnel, username, password
                assert "tunnel" in args.keys(), "Please make sure \'tunnel\' in your keys"
                assert "username" in args.keys(), "Please make sure \'username\' in your keys"
                assert "password" in args.keys(), "Please make sure \'password\' in your keys"
                self.proxy_list = Kuaidaili(args["tunnel"], args["username"], args["password"])
            else:
                self.proxy_id = 0
                self.proxy_list = self._update_proxy()
        else:
            self.proxy_list = []

    def _get_proxy(self):
        if self.use_proxy:
            if "kuaidaili" in self.country:
                proxy = self.proxy_list.get_kuaidaili_tunnel_proxy()
                return proxy
            elif len(self.proxy_list) >0:
                proxy = self.proxy_list[self.proxy_id]
                self.proxy_id += 1
                if self.proxy_id == len(self.proxy_list):
                    self.proxy_id = 0
                return proxy
        else:
            return None

    def _update_proxy(self):
        if "china" in self.country or "China" in self.country:
            return get_china_free_proxy(self.proxy_pages)
        else:
            return get_us_free_proxy(self.proxy_pages)

    def _request_get(self, url, headers = None, verify = None, params = None):
        if headers is None:
            headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0"
            }
        max_retry = self.max_retry
        proxies = self._get_proxy()
        for _ in range(max_retry):
            try:
                response = requests.get(url = url, proxies = proxies, headers = headers, verify = verify, params = params)
                if response.status_code == 200:
                    break
            except:
                response = None

        if response is not None and response.status_code != 200:
            response = None

        return response

    def _request_post(self, url, headers, json):
        max_retry = self.max_retry
        proxies = self._get_proxy()
        for _ in range(max_retry):
            try:
                response = requests.post(url = url, headers = headers, json = json, proxies = proxies)
                if response.status_code == 200:
                    break
            except:
                response = None

        if response is not None and response.status_code != 200:
            response = None

        return response

class News_Downloader(FinNLP_Downloader):

    def __init__(self, args = {}):
        super().__init__(args)
        pass

    def download_date_range(self, start_date, end_date, stock = None):
        pass

    def download_streaming(self, stock = None):
        pass

    def clean_data(self):
        pass

    def _gather_one_part(self, date, stock = None, delay = 0.1):
        pass

    def _gather_content(self):
        pass

from tqdm import tqdm
from lxml import etree
import pandas as pd
import requests
import finnhub
import time
import json

class Finnhub_Date_Range(News_Downloader):
    def __init__(self, args = {}):
        super().__init__(args)
        assert "token" in args.keys(), "Please input your finnhub token. Avaliable at https://finnhub.io/dashboard"
        self.finnhub_client = finnhub.Client(api_key=args["token"])

    def download_date_range_stock(self, start_date, end_date, stock = "AAPL"):
        self.date_list = pd.date_range(start_date,end_date)
        self.dataframe = pd.DataFrame()

        days_each_time = 4
        date_list = self.date_list
        # cal total lenth
        if len(date_list)%days_each_time == 0:
            total = len(date_list)//days_each_time
        else:
            total = len(date_list)//days_each_time+1

        with tqdm(total=total, desc= "Downloading Titles") as bar:
            while len(date_list):
                tmp_date_list = date_list[:days_each_time]
                date_list = date_list[days_each_time:]
                tmp_start_date = tmp_date_list[0].strftime("%Y-%m-%d")
                tmp_end_date = tmp_date_list[-1].strftime("%Y-%m-%d")
                res = self._gather_one_part(tmp_start_date,tmp_end_date,stock = stock )
                self.dataframe = pd.concat([self.dataframe,res])
                bar.update(1)

        # res  = self.finnhub_client.company_news(stock, _from=start_date, to=end_date)
        self.dataframe.datetime = pd.to_datetime(self.dataframe.datetime,unit = "s")
        self.dataframe = self.dataframe.reset_index(drop = True)

    def _gather_one_part(self, start_date, end_date, stock = "AAPL", delay = 1):
        res = self.finnhub_client.company_news(stock, _from=start_date, to=end_date)
        time.sleep(delay)
        return pd.DataFrame(res)

    def gather_content(self, delay = 0.01):
        pbar = tqdm(total = self.dataframe.shape[0], desc= "Gathering news contents")
        self.dataframe["content"] = self.dataframe.apply(lambda x:self._gather_content_apply(x, pbar, delay), axis = 1)

    def _gather_content_apply(self,x, pbar, delay = 0.01):
        time.sleep(delay)
        url = x.url
        source = x.source
        response = self._request_get(url = url)
        # response = self._request_get(url= url, headers= headers)
        pbar.update(1)
        if response is None:
            return "Connection Error"
        else:
            page = etree.HTML(response.text)

        try:
            # Yahoo Finance
            if source == "Yahoo":
                page = page.xpath("/html/body/div[3]/div[1]/div/main/div[1]/div/div/div/div/article/div/div/div/div/div/div[2]/div[4]")
                content = page[0].xpath(".//text()")
                content = "\n".join(content)
                return content

            # Reuters
            elif source == "Reuters":
                page = page.xpath("/html/body/div[1]/div[3]/div/main/article/div[1]/div[2]/div/div/div[2]")
                content = page[0].xpath(".//text()")
                content = "\n".join(content)
                return content

            # SeekingAlpha
            elif source == "SeekingAlpha":
                page = page.xpath("/html/body/div[2]/div/div[1]/main/div/div[2]/div/article/div/div/div[2]/div/section[1]/div/div/div")
                content = page[0].xpath(".//text()")
                content = "\n".join(content)
                return content

            # PennyStocks
            elif source == "PennyStocks":
                page = page.xpath("/html/body/div[3]/div/div[1]/div/div/div/main/article/div[2]/div[2]/div")
                content = page[0].xpath(".//text()")
                content = "\n".join(content)
                return content

            # MarketWatch
            elif source == "MarketWatch":
                page = page.xpath('//*[@id="js-article__body"]')
                content = page[0].xpath(".//text()")
                content = "".join(content)
                while "  " in content:
                    content = content.replace("  ", " ")
                while "\n \n"in content:
                    content = content.replace("\n \n", " ")
                while "\n  "in content:
                    content = content.replace("\n  ", " ")
                return content

            # Seeking Alpha
            elif source == "Seeking Alpha":
                # first get Seeking Alpha URL
                page = page.xpath('/html/body/div[5]/div[2]/section[1]/article[2]/div/div[2]/p/a/@href')
                url_new = page[0]
                response = self._request_get(url= url_new)
                if response is None:
                    return "Connection Error"
                else:
                    page = etree.HTML(response.text)

                content = page[0].xpath(".//text()")
                content = "\n".join(content)
                return content

            # Alliance News
            elif source == "Alliance News":
                page = page.xpath('//*[@id="comtext"]')
                content = page[0].xpath(".//text()")
                content = [c for c in content if not str(c).startswith("\r\n")]
                content = "\n".join(content)
                return content

            # Thefly.com
            elif source == "Thefly.com":
                page = page.xpath('/html/body/div[5]/div[2]/section[1]/article[2]/div/div[2]/p/a/@href')
                url_new = page[0]
                response = self._request_get(url= url_new, verify= False)
                if response is None:
                    return "Connection Error"
                else:
                    page = etree.HTML(response.text)

                page = page.xpath('/html/body/div[2]/div/div/div/div/div[2]/div[2]//text()')
                # content = page[0].xpath(".//text()")
                # content = [c for c in content if not str(c).startswith("\r\n")]
                content = "\n".join(page)
                content = content.replace("\r\n","")

                return content

            # TalkMarkets
            elif source == "TalkMarkets":
                return "Not supported yet"

            # CNBC
            elif source == "CNBC":
                page = page.xpath('/html/body/div[3]/div/div[1]/div[3]/div/div/div/div[3]/div[1]/div[2]/div[3]//text()')
                content = "\n".join(page)

                return content

            # GuruFocus
            elif source == "GuruFocus":
                page = page.xpath('/html/body/div[5]/div[2]/section[1]/article[2]/div/div[2]/p/a/@href')
                url_new = page[0]
                response = self._request_get(url= url_new)
                if response is None:
                    return "Connection Error"
                else:
                    page = etree.HTML(response.text)

                page = page.xpath('/html/body/div[1]/div/section/section/main/section/main/div[1]/div/div/div[1]/div[2]/div//text()')
                page_new = []
                for c in page:
                    while "\n"  in c:
                        c = c.replace("\n","")
                    while "  "in c:
                        c = c.replace("  ","")

                    page_new.append(c)

                content = "\n".join(page_new)

                return content

            # InvestorPlace
            elif source == "InvestorPlace":
                page = page.xpath('/html/body/div[5]/div[2]/section[1]/article[2]/div/div[2]/p/a/@href')
                url_new = page[0]
                response = self._request_get(url= url_new)
                if response is None:
                    return "Connection Error"
                else:
                    page = etree.HTML(response.text)
                    page = page.xpath('//script[@type="application/ld+json"]')[1]
                    content = page.xpath(".//text()")
                    content = json.loads(content[0])
                    content = content["articleBody"]

                    return content

            # TipRanks
            elif source == "TipRanks":
                page = page.xpath('/html/body/div[5]/div[2]/section[1]/article[2]/div/div[2]/p/a/@href')
                url_new = page[0]
                response = self._request_get(url= url_new)
                if response is None:
                    return "Connection Error"
                else:
                    page = etree.HTML(response.text)
                    # /html/body/div[1]/div[2]/div[5]/div[2]/div[2]/div/div[6]/div/article/p[1]/p
                    page = page.xpath('/html/body/div[1]/div[1]/div[4]/div[2]/div[2]/div[1]/div[6]//text()')
                    # content = page[0].xpath('.//text()')
                    page = [p.replace("\n","") for p in page]
                    content = "".join(page)
                    return content

            else:
                return "Not supported yet"

        except:
            return "Error"


In [16]:
finnhub_client = finnhub.Client(api_key=finnhub_api_key)

print(finnhub_client.company_news('AAPL', _from="2024-02-01", to="2024-03-10"))

quote = finnhub_client.quote('AAPL') #

peers = finnhub_client.company_peers('AAPL')

[{'category': 'company', 'datetime': 1709481000, 'headline': "Here's How Much You'd Have If You Invested $10,000 in Apple Stock When Warren Buffett First Bought It", 'id': 126282850, 'image': 'https://g.foolcdn.com/editorial/images/767305/aapl.png', 'related': 'AAPL', 'source': 'Yahoo', 'summary': "Following in Buffett's footsteps would have paid off nicely with Apple.", 'url': 'https://finnhub.io/api/news?id=d6e519c7bddf511b60a4afe0778c76e1cf743ce472f5004b511e4fc5398e729f'}, {'category': 'company', 'datetime': 1709480023, 'headline': 'Banks call for crackdown on Apple over claims it stockpiles British spending data', 'id': 126282851, 'image': 'https://www.telegraph.co.uk/content/dam/business/2024/03/03/TELEMMGLPICT000070592735_17094782863820_trans_NvBQzQNjv4BqAWXnVVRQmej3264Ho6DEuGKrcxidEJqOo_UlPWKcDWU.jpeg?impolicy=logo-overlay', 'related': 'AAPL', 'source': 'Yahoo', 'summary': 'Major banks have called for a crackdown on Apple over claims the tech giant’s access to millions of iPhone

{'c': 179.66,
 'd': -1.09,
 'dp': -0.603,
 'h': 180.53,
 'l': 177.38,
 'o': 179.55,
 'pc': 180.75,
 't': 1709326801}

In [19]:
from datetime import date, timedelta

finnhub_client = finnhub.Client(api_key=finnhub_api_key)

def get_finnhub_info(ticker):

  # time range: last 10 days, but taking tomorrow's date as 'end' to capture international markets
  today = date.today()
  tomorrow = today + timedelta(days=1)
  ten_days_ago = today - timedelta(days=10)
  tomorrow_formatted = tomorrow.strftime('%Y-%m-%d')
  ten_days_ago_formatted = ten_days_ago.strftime('%Y-%m-%d')

  news = finnhub_client.company_news(ticker, _from=ten_days_ago_formatted, to=tomorrow_formatted)

  quote = finnhub_client.quote(ticker)

  peers = finnhub_client.company_peers(ticker)

  return {"news": news, "quote": quote, "peers": peers}

In [20]:
get_finnhub_info('AAPL')

{'news': [{'category': 'company',
   'datetime': 1709481000,
   'headline': "Here's How Much You'd Have If You Invested $10,000 in Apple Stock When Warren Buffett First Bought It",
   'id': 126282850,
   'image': 'https://g.foolcdn.com/editorial/images/767305/aapl.png',
   'related': 'AAPL',
   'source': 'Yahoo',
   'summary': "Following in Buffett's footsteps would have paid off nicely with Apple.",
   'url': 'https://finnhub.io/api/news?id=d6e519c7bddf511b60a4afe0778c76e1cf743ce472f5004b511e4fc5398e729f'},
  {'category': 'company',
   'datetime': 1709480023,
   'headline': 'Banks call for crackdown on Apple over claims it stockpiles British spending data',
   'id': 126282851,
   'image': 'https://www.telegraph.co.uk/content/dam/business/2024/03/03/TELEMMGLPICT000070592735_17094782863820_trans_NvBQzQNjv4BqAWXnVVRQmej3264Ho6DEuGKrcxidEJqOo_UlPWKcDWU.jpeg?impolicy=logo-overlay',
   'related': 'AAPL',
   'source': 'Yahoo',
   'summary': 'Major banks have called for a crackdown on Apple o

### Quote

c
Current price

d
Change

dp
Percent change

h
High price of the day

l
Low price of the day

o
Open price of the day

pc
Previous close price

In [12]:
finnhub_client.stock_candles('AAPL', 'D', 1590988249, 1591852249)

start_date = "2023-01-01"
end_date = "2023-01-03"
config = {
    "use_proxy": "us_free",    # use proxies to prvent ip blocking
    "max_retry": 5,
    "proxy_pages": 5,
    "token": finnhub_api_key  # Available at https://finnhub.io/dashboard
}

news_downloader = Finnhub_Date_Range(config)                      # init
news_downloader.download_date_range_stock(start_date,end_date)    # Download headers
news_downloader.gather_content()                                  # Download contents
df = news_downloader.dataframe
selected_columns = ["headline", "content"]
df[selected_columns].head(10)

FinnhubAPIException: FinnhubAPIException(status_code: 403): You don't have access to this resource.

In [4]:
# Finnhub (Yahoo Finance, Reuters, SeekingAlpha, CNBC...)
from finnlp.data_sources.news.finnhub_date_range import Finnhub_Date_Range

start_date = "2023-01-01"
end_date = "2023-01-03"
config = {
    "use_proxy": "us_free",    # use proxies to prvent ip blocking
    "max_retry": 5,
    "proxy_pages": 5,
    "token": "YOUR_FINNHUB_TOKEN"  # Available at https://finnhub.io/dashboard
}

news_downloader = Finnhub_Date_Range(config)                      # init
news_downloader.download_date_range_stock(start_date,end_date)    # Download headers
news_downloader.gather_content()                                  # Download contents
df = news_downloader.dataframe
selected_columns = ["headline", "content"]
df[selected_columns].head(10)

ModuleNotFoundError: No module named 'finnlp'